1. Recorte das tabelas pertinentes do banco de dados

In [2]:
import sqlite3

# Caminho do banco de dados original
original_db_path = "../data/DiscursosSenadores_02_05_2025.sqlite"
# Caminho do novo banco de dados
new_db_path = "./DiscursosSenadores_02_05_2025_analisado.sqlite"

# Conectar ao banco de dados original
orig_conn = sqlite3.connect(original_db_path)
orig_cursor = orig_conn.cursor()

# Conectar ao novo banco de dados (será criado se não existir)
new_conn = sqlite3.connect(new_db_path)
new_cursor = new_conn.cursor()

# Lista das tabelas a serem copiadas
tables_to_copy = ["Discursos", "Senadores"]


# Função para criar e copiar tabelas específicas
def copy_table(cursor, new_cursor, table_name):
    # Obter schema da tabela
    cursor.execute(f"SELECT sql FROM sqlite_master WHERE type='table' AND name='{table_name}';")
    create_table_sql = cursor.fetchone()[0]

    # Criar a tabela no novo banco de dados
    new_cursor.execute(create_table_sql)

    # Copiar dados
    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()

    # Identificar colunas da tabela
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = [col[1] for col in cursor.fetchall()]
    placeholders = ", ".join(["?"] * len(columns))
    new_cursor.executemany(
        f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({placeholders})", rows
    )


# Processar cada tabela
for table in tables_to_copy:
    copy_table(orig_cursor, new_cursor, table)

# Commit e fechar conexões
new_conn.commit()
orig_conn.close()
new_conn.close()

print(f"Novo banco de dados criado em: {new_db_path}")


Novo banco de dados criado em: ./DiscursosSenadores_02_05_2025_analisado.sqlite


2. Vacuum dos arquivos sqlite

In [3]:
import sqlite3

# Arquivos SQLite a serem otimizados
files_to_vacuum = [
    "./DiscursosSenadores_02_05_2025_analisado.sqlite",
    "../data/DiscursosSenadores_02_05_2025.sqlite"
]

# Executa VACUUM em cada arquivo
for db_file in files_to_vacuum:
    print(f"Executando VACUUM no arquivo: {db_file}")
    conn = sqlite3.connect(db_file)
    conn.execute("VACUUM;")  # Executa o comando VACUUM
    conn.close()
    print(f"VACUUM concluído para {db_file}")

print("Processo completo!")


Executando VACUUM no arquivo: ./DiscursosSenadores_02_05_2025_analisado.sqlite
VACUUM concluído para ./DiscursosSenadores_02_05_2025_analisado.sqlite
Executando VACUUM no arquivo: ../data/DiscursosSenadores_02_05_2025.sqlite
VACUUM concluído para ../data/DiscursosSenadores_02_05_2025.sqlite
Processo completo!
